Concept 1 :-  Generating 3 different matrix with only one single matrix data

In [2]:
import torch
import numpy
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Concept1(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.embdim = emb_dim
        self.k = nn.Linear(emb_dim,emb_dim)
        self.q = nn.Linear(emb_dim,emb_dim)
        self.v = nn.Linear(emb_dim,emb_dim)

    def forward(self,x):
        k = self.k(x)
        q = self.q(x)
        v = self.v(x)
        print(k)
        print(q)
        print(v)

a = torch.randn(6, 5)
b = torch.tensor(a, requires_grad=True)

c1 = Concept1(emb_dim=b.shape[-1])
c1(b)

tensor([[-1.2205, -0.5107, -0.5776,  1.0154, -0.3387],
        [ 0.8051, -0.1997, -0.1864,  0.2760,  0.7929],
        [-0.8045,  0.1211, -0.4704,  0.7659, -0.1839],
        [ 0.3040, -0.7501, -0.5700,  0.7559,  0.4723],
        [ 0.2932, -0.6044,  0.1543,  0.0408,  0.6389],
        [-0.3585, -0.7708, -0.0678,  0.3714,  0.2312]],
       grad_fn=<AddmmBackward0>)
tensor([[-1.2144, -0.2434,  1.0975,  0.2637, -1.3388],
        [ 1.0139, -0.6744, -0.2186, -1.0284, -0.1813],
        [-0.7618,  0.1986,  0.8440, -0.4387,  0.1197],
        [-0.2579, -0.4177,  0.2553,  0.0783, -0.9059],
        [ 0.5311, -0.5970,  0.1204, -0.2205, -0.8894],
        [-0.4706, -0.2649,  0.6005,  0.4256, -1.1617]],
       grad_fn=<AddmmBackward0>)
tensor([[ 1.2427,  0.6524,  0.3455,  0.0935, -1.1860],
        [-0.2993, -1.3685, -0.0400,  0.5771, -0.5586],
        [ 0.7445,  0.5836, -0.7850, -0.6655,  0.1324],
        [ 0.0342, -0.5207,  0.4102,  0.2069, -0.6501],
        [-0.2358, -0.8353,  0.1298,  0.4162, -0.7673

C:\Users\goura\AppData\Local\Temp\ipykernel_18064\3350324195.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.tensor(a, requires_grad=True)


self attention

concept :-
matmul(v,softmax(matrixmul(k,qT)/scaler))

In [27]:
class SelfAttention(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.embdim = emb_dim
        self.k = nn.Linear(emb_dim,emb_dim)
        self.q = nn.Linear(emb_dim,emb_dim)
        self.v = nn.Linear(emb_dim,emb_dim)

    def forward(self,x):
        k = self.k(x)
        q = self.q(x)
        v = self.v(x)
        print(k.shape)
        scaler = self.embdim
        s1 = q @ k.transpose(-1,-2)/(scaler**0.5)
        att = F.softmax(s1,dim=-1)
        out = att @ v
        return out, att

a = torch.randn(2, 6, 8)
b = torch.tensor(a, requires_grad=True)
B, T, E = b.shape
attention = SelfAttention(emb_dim=b.shape[-1])
out , att = attention(b)
# print(out)
# print(att)


torch.Size([2, 6, 5])


C:\Users\goura\AppData\Local\Temp\ipykernel_18064\3045246291.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.tensor(a, requires_grad=True)


multihead attention

Concept:-
let say we have embedding dim as (2,5,8) then our responce should also contain 5 words as rows and 8 dim embedding of each word
concept is we will adjust of the initial 
        self.k = nn.Linear(emb_dim,emb_dim)
        self.q = nn.Linear(emb_dim,emb_dim)
        self.v = nn.Linear(emb_dim,emb_dim)
        in such a way that in the end of multihead attention process we satisfy the above condition
cases
1) (B, T, E) → (B, T, H, D)
2) transpose (B, T, H, D) → (B, H, T, D) this will allow us to process all head result together


In [ ]:
class Multihead1(nn.Module):
    def __init__(self,emb_dim,n_heads):
        super().__init__()
        assert emb_dim % n_heads == 0, "Embedding must divide heads"
        self.emb_dim = emb_dim
        self.n_heads = n_heads
        self.head_dim = emb_dim // n_heads
        self.k = nn.Linear(emb_dim,emb_dim)
        self.q = nn.Linear(emb_dim,emb_dim)
        self.v = nn.Linear(emb_dim,emb_dim)
        self.W_o = nn.Linear(emb_dim, emb_dim)
    def forward(self,x):
        B, T, E = x.shape
        q = self.k(x)
        k = self.q(x)
        v = self.v(x)
        q = q.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        att = q @ k.transpose(-2, -1)
        att = att / (self.head_dim ** 0.5)
        att = F.softmax(att, dim=-1)
        context = att @ v
        context = context.transpose(1, 2)
        print(context.shape)
        context = context.reshape(B, T, E)
        print(context.shape)
        out = self.W_o(context)
        return out
    

class MultiHeadAttention(nn.Module):
    def __init__(self,emb_d,n_head):
        super().__init__()
        assert emb_d%n_head == 0 , "Embedding must divide head"
        self.nhead = n_head
        self.dhead = emb_d//n_head
        self.wk = nn.Linear(emb_d,emb_d*n_head)
        self.wq = nn.Linear(emb_d,emb_d*n_head)
        self.wv = nn.Linear(emb_d,emb_d*n_head)
        self.wo = nn.Linear(emb_d*n_head,emb_d)
    
    def forward(self,x):
        B,T,E = x.shape
        # (2,4,8) -> (2,4,32) -> (2,4,4,8)
        k = self.wk(x).view(B,T,self.nhead,E).transpose(1,2)
        q = self.wq(x).view(B,T,self.nhead,E).transpose(1,2)
        v = self.wv(x).view(B,T,self.nhead,E).transpose(1,2)
        att = q@k.transpose(-2,-1)
        att = att / (self.dhead ** 0.5)
        att = F.softmax(att, dim=-1)
        context = att @ v
        context = context.transpose(1, 2)
        # (2,4,4,8)->(2,4,32)
        context = context.reshape(B, T, E*self.nhead)
        print(context.shape)
        # (2,4,32)->(2,4,8)
        out = self.wo(context)
        return out
    
a = torch.randn(1,4,4)
m =MultiHeadAttention(a.shape[-1],4)
m(a)




torch.Size([1, 4, 16])


tensor([[[-0.1677,  0.2940,  0.4168, -0.0701],
         [-0.1945,  0.2490,  0.4724, -0.0158],
         [-0.1953,  0.3551,  0.3990, -0.0788],
         [-0.1599,  0.2814,  0.3987, -0.0905]]], grad_fn=<ViewBackward0>)

cross attention

positional encoding

In [20]:
import math

def function(pos,i,d):
    if(i%2==0):
        return math.sin(pos/10000**(2*i/d))
    return math.cos(pos/10000**(2*i/d))

def Positional(t):
    for pos,block in enumerate(t):
        for i,_ in enumerate(block):
            a = function(pos,i,len(block))
            block[i]+=a
    return t

a = torch.randn(6, 5)
b = torch.tensor(a, requires_grad=True)

C:\Users\goura\AppData\Local\Temp\ipykernel_18064\33762896.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.tensor(a, requires_grad=True)
